In [10]:
# Various torch packages
import torch
import torch.nn as nn
import torch.nn.functional as F

# torchvision
from torchvision import datasets, transforms

# ------------------------
# get up one directory 
import sys, os
sys.path.append(os.path.abspath('../'))
# ------------------------

# custom packages
import matplotlib.pyplot as plt
import models.aux_funs as maf
import optimizers as op
import regularizers as reg
import train
import math
import utils.configuration as cf
import utils.datasets as ud
from models.mnist_conv import mnist_conv
from utils.visualise import visualize_layer_weights

# Fix the random seed

In [11]:
random_seed = 2
cf.seed_torch(random_seed)

# Configure the experiment

In [12]:
import itertools

l_0s = [0.03, 0.04, 0.05]
l_1s = [0.03, 0.04, 0.05]
deltas = [1, 0.9, 0.8]

combinations = list(itertools.product(l_0s, l_1s, deltas))

def print_combination(combination):
    print(f'Combination: l_0 = {combination[0]}, l_1 = {combination[1]}, delta = {combination[2]}')

base_args = {#
    # data specification
    'data_file':"../../Data",'train_split':0.95, 'data_set':"Fashion-MNIST", 'download':True,
    # cuda
    'use_cuda':True, 'num_workers':4, 'cuda_device':0, 'pin_memory':True,
    #
    'epochs':100,
    # optimizer
    'delta':1.0, 'lr':0.001, 'lamda_0':0.4, 'lamda_1':0.4, 'optim':"AdaBreg", 'conv_group':True,
    'beta':0.0,
    # initialization
    'sparse_init':0.01, 'r':[1,1,1],
    # misc
    'random_seed':random_seed, 'eval_acc':True,
}

args = []
for (l_0, l_1, delta) in combinations:
    conf_args = base_args.copy()
    conf_args['lamda_0'] = l_0
    conf_args['lamda_1'] = l_1
    conf_args['delta'] = delta
    args.append(conf_args)
    
confs = [cf.Conf(**conf_args) for conf_args in args]


# Initiate the models

In [13]:
kwargs = [{'mean':conf.data_set_mean, 'std':conf.data_set_std} for conf in confs]   

models = [mnist_conv(**kwarg) for kwarg in kwargs]

best_models = [train.best_model(mnist_conv(**kwarg).to(conf.device)) for kwarg, conf in zip(kwargs, confs)]

# Weight initializations

In [14]:
   
def init_weights(conf, model):
    # sparsify
    maf.sparse_bias_uniform_(model, 0,conf.r[0])
    maf.sparse_bias_uniform_(model, 0,conf.r[0],ltype=torch.nn.Conv2d)
    maf.sparse_weight_normal_(model, conf.r[1])
    maf.sparse_weight_normal_(model, conf.r[2],ltype=torch.nn.Conv2d)
    #
    maf.sparsify_(model, conf.sparse_init, ltype = nn.Conv2d, conv_group=conf.conv_group)
    model = model.to(conf.device)
    
    return model


initialized_models = [init_weights(conf, model) for conf, model in zip(confs, models)]

# Optimizer

In [15]:
def init_opt(conf, model):
    # -----------------------------------------------------------------------------------
    # Get access to different model parameters
    # -----------------------------------------------------------------------------------
    weights_conv = maf.get_weights_conv(model)
    weights_linear = maf.get_weights_linear(model)
    biases = maf.get_bias(model)
    
    # -----------------------------------------------------------------------------------
    # Initialize optimizer
    # -----------------------------------------------------------------------------------
    if conf.conv_group:
        reg2 = reg.reg_l1_l2_conv(lamda=conf.lamda_0)
    else:
        reg2 = reg.reg_l1(lamda=conf.lamda_0)
    
    if conf.optim == "SGD":
        opt = torch.optim.SGD(model.parameters(), lr=conf.lr, momentum=conf.beta)
    elif conf.optim == "LinBreg":
        opt = op.LinBreg([{'params': weights_conv, 'lr' : conf.lr, 'reg' : reg2, 'momentum':conf.beta,'delta':conf.delta},
                          {'params': weights_linear, 'lr' : conf.lr, 'reg' : reg.reg_l1(lamda=conf.lamda_1), 'momentum':conf.beta,'delta':conf.delta},
                          {'params': biases, 'lr': conf.lr, 'momentum':conf.beta}])
    elif conf.optim == "ProxSGD":
        opt = op.ProxSGD([{'params': weights_conv, 'lr' : conf.lr, 'reg' : reg2, 'momentum':conf.beta,'delta':conf.delta},
                          {'params': weights_linear, 'lr' : conf.lr, 'reg' : reg.reg_l1(lamda=conf.lamda_1), 'momentum':conf.beta,'delta':conf.delta},
                          {'params': biases, 'lr': conf.lr, 'momentum':conf.beta}])            
    elif conf.optim == "AdaBreg":
        opt = op.AdaBreg([{'params': weights_conv, 'lr' : conf.lr, 'reg' : reg2,'delta':conf.delta},
                           {'params': weights_linear, 'lr' : conf.lr, 'reg' : reg.reg_l1(lamda=conf.lamda_1),'delta':conf.delta},
                           {'params': biases, 'lr': conf.lr}])
    elif conf.optim == "L1SGD":
        def weight_reg(model):
            reg1 =  reg.reg_l1(lamda=conf.lamda_1)
        
            loss1 = reg1(model.layers2[0].weight) + reg1(model.layers2[2].weight)
            loss2 = reg2(model.layers1[0].weight) + reg2(model.layers1[3].weight)
            return loss1 + loss2
        
        conf.weight_reg = weight_reg
        
        opt = torch.optim.SGD(model.parameters())
    elif conf.optim == "Adam":
        opt = torch.optim.Adam(model.parameters())
    else:
        raise ValueError("Unknown Optimizer specified")

    # learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=5,threshold=0.01)
    
    return opt, scheduler

# Dataset

In [16]:
# train_loader, valid_loader, test_loader = ud.get_data_set(conf)

data = [ud.get_data_set(conf) for conf in confs]

# History and Runs

In [17]:
# Initialize history
tracked = ['loss', 'node_sparse']
# train_hist = {}
# val_hist = {}

train_hists = [{} for conf in confs]
val_hists = [{} for conf in confs]

In [ ]:
for index in range(len(confs)):
    combination = combinations[index]
    print_combination(combination)
    conf = confs[index]
    model = initialized_models[index]
    best_model = best_models[index]
    opt, scheduler = init_opt(conf, model)
    train_loader, valid_loader, test_loader = data[index]
    train_hist = train_hists[index]
    val_hist = val_hists[index]

    # train the model
    for epoch in range(conf.epochs):
        print(25*"<>")
        print(50*"|")
        print(25*"<>")
        print('Epoch:', epoch)
    
        # ------------------------------------------------------------------------
        # train step, log the accuracy and loss
        # ------------------------------------------------------------------------
        train_data = train.train_step(conf, model, opt, train_loader)
    
        # update history
        for key in tracked:
            if key in train_data:
                var_list = train_hist.setdefault(key, [])
                var_list.append(train_data[key])           
    
        # ------------------------------------------------------------------------
        # validation step
        val_data = train.validation_step(conf, model, opt, valid_loader)
    
        # update validation history
        for key in tracked:
            if key in val_data:
                var = val_data[key]
                if isinstance(var, list):
                    for i, var_loc in enumerate(var):
                        key_loc = key+"_" + str(i)
                        var_list = val_hist.setdefault(key_loc, [])
                        val_hist[key_loc].append(var_loc)
                else:
                    var_list = val_hist.setdefault(key, [])
                    var_list.append(var)   
    
    
        scheduler.step(train_data['loss'])
        print("Learning rate:",opt.param_groups[0]['lr'])
        best_model(train_data['acc'], val_data['acc'], model=model)
    

Combination: l_0 = 0.03, l_1 = 0.03, delta = 1
<><><><><><><><><><><><><><><><><><><><><><><><><>
||||||||||||||||||||||||||||||||||||||||||||||||||
<><><><><><><><><><><><><><><><><><><><><><><><><>
Epoch: 0
--------------------------------------------------
Train Accuracy: 0.14221052631578948
Train Loss: 129.232177734375
--------------------------------------------------
Validation Accuracy: 0.5153333333333333
Non-zero kernels: 0.024278846153846154
Linear sparsity: 0.9561623549323017
Overall sparsity: 0.5461134240454915
Node sparsity: [1.0, 1.0]
Regularization values per group: [1.5526994705200194, 143.642137298584, 0.0]
Learning rate: 0.001
<><><><><><><><><><><><><><><><><><><><><><><><><>
||||||||||||||||||||||||||||||||||||||||||||||||||
<><><><><><><><><><><><><><><><><><><><><><><><><>
Epoch: 1
--------------------------------------------------
Train Accuracy: 0.682859649122807
Train Loss: 51.47278678417206
--------------------------------------------------
Validation Accuracy:

# Training

In [29]:
# # -----------------------------------------------------------------------------------
# # Reinit weigts and the corresponding optimizer
# # -----------------------------------------------------------------------------------
# model = init_weights(conf, model)
# opt, scheduler = init_opt(conf, model)
# 
# # -----------------------------------------------------------------------------------
# # train the model
# # -----------------------------------------------------------------------------------
# for epoch in range(conf.epochs):
#     print(25*"<>")
#     print(50*"|")
#     print(25*"<>")
#     print('Epoch:', epoch)
# 
#     # ------------------------------------------------------------------------
#     # train step, log the accuracy and loss
#     # ------------------------------------------------------------------------
#     train_data = train.train_step(conf, model, opt, train_loader)
# 
#     # update history
#     for key in tracked:
#         if key in train_data:
#             var_list = train_hist.setdefault(key, [])
#             var_list.append(train_data[key])           
# 
#     # ------------------------------------------------------------------------
#     # validation step
#     val_data = train.validation_step(conf, model, opt, valid_loader)
# 
#     # update validation history
#     for key in tracked:
#         if key in val_data:
#             var = val_data[key]
#             if isinstance(var, list):
#                 for i, var_loc in enumerate(var):
#                     key_loc = key+"_" + str(i)
#                     var_list = val_hist.setdefault(key_loc, [])
#                     val_hist[key_loc].append(var_loc)
#             else:
#                 var_list = val_hist.setdefault(key, [])
#                 var_list.append(var)   
# 
# 
#     scheduler.step(train_data['loss'])
#     print("Learning rate:",opt.param_groups[0]['lr'])
#     best_model(train_data['acc'], val_data['acc'], model=model)

In [ ]:
# Save Model
for index, combi in enumerate(combinations):
    print_combination(combi)
    best_model = best_models[index]
    conf = confs[index]
    test_loader = data[index][2]
    
    model = best_model.best_model
    
    torch.save(model.state_dict(), f'../checkpoints/hyperparameters/mnist_conv_{combi[0]}_{combi[1]}_{combi[2]}.pth')
    
    print('Evaluate')
    model.eval()
    print('Test')
    train.test(conf, model, test_loader)
    print('-'*50)